In [19]:
from pathlib import Path
import sys

root = str(Path().resolve().parents[0])
if root not in sys.path:
    sys.path.append(root)

from src.utils import ROOT, DATA_DIR, EXPERIMENTS_DIR, OUTPUT_DIR, DATA_PATHS

for path in [ROOT, DATA_DIR, EXPERIMENTS_DIR, OUTPUT_DIR]:
    if str(path.resolve()) not in sys.path:
        sys.path.append(str(path.resolve()))

print(*sys.path, sep="\n")

/usr/lib/python312.zip
/usr/lib/python3.12
/usr/lib/python3.12/lib-dynload

/home/ryanwtsai/repos/ml_ai_portfolio/notes/example_code/venv/lib/python3.12/site-packages
/home/ryanwtsai/repos/ml_ai_portfolio/titanic
/home/ryanwtsai/repos/ml_ai_portfolio/titanic/dataset
/home/ryanwtsai/repos/ml_ai_portfolio/titanic/experiments
/home/ryanwtsai/repos/ml_ai_portfolio/titanic/outputs


In [22]:
from importlib import reload
import src
# import experiment_config

reload(src)
reload(src.utils)
reload(src.custom_transformers)
reload(src.custom_model_selection)
reload(src.experiment_runner)
reload(src.config_parser)
# reload(experiment_config)

<module 'src.config_parser' from '/home/ryanwtsai/repos/ml_ai_portfolio/titanic/src/config_parser.py'>

In [29]:
import yaml
from src.config import build_estimators

with open(ROOT / "scripts/experiment_config.yaml", "r") as f:
    config = yaml.safe_load(f)

config

estimators = build_estimators(config)

In [32]:
from src.utils import load_titanic_data

pipeline = estimators["ml_pipe"]
X, y, _ = load_titanic_data()

pipeline.fit(X, y)

Pipeline(steps=[('data_prep',
                 DynamicDataPrepPipeline(age_imputer_model=RandomForestRegressor(max_depth=20),
                                         extract_deck=True, extract_fam=True,
                                         extract_sexpclassage=True,
                                         extract_title=True,
                                         numeric_columns=['Age', 'Pclass',
                                                          'Fare'],
                                         numeric_transformations={'Fare': Pipeline(steps=[('bin',
                                                                                           KBinsDiscretizer(encode='ordinal',
                                                                                                            n_bins=4)),
                                                                                          ('scale',
                                                                                           MinMaxScaler())]),
                                                                  'Pclass': MinMaxScaler()},
                                         ordinal_columns=['Sex'])),
                ('model',
                 RandomForestClassifier(max_depth=20, random_state=0))])

In [36]:
pipeline.named_steps["data_prep"].pipeline_

Pipeline(steps=[('extractor',
                 Pipeline(steps=[('fare_imputer', FareImputer()),
                                 ('fam', FamilySizeExtractor()),
                                 ('title', TitleExtractor()),
                                 ('deck', DeckExtractor()),
                                 ('age_imputer',
                                  AgeImputer(model=RandomForestRegressor(max_depth=20))),
                                 ('sexpclassage', SexPclassAgeExtractor())])),
                ('col_tf',
                 ColumnTransformer(transformers=[('num_Fare',
                                                  Pipeline(steps=[('bin',
                                                                   K...
                                                                   OrdinalEncoder(categories=[['U',
                                                                                               'G',
                                                                                               'F',
                                                                                               'E',
                                                                                               'D',
                                                                                               'C',
                                                                                               'B',
                                                                                               'A']])),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['Deck']),
                                                 ('onehot_Sex',
                                                  OneHotEncoder(categories=[['male',
                                                                             'female']]),
                                                  ['Sex']),
                                                 ('onehot_Title',
                                                  OneHotEncoder(categories=[['Mr',
                                                                             'Miss',
                                                                             'Mrs',
                                                                             'Master',
                                                                             'Dr',
                                                                             'Rev',
                                                                             'Unknown']],
                                                                handle_unknown='ignore'),
                                                  ['Title']),
                                                 ('onehot_SexPclassAge',
                                                  OneHotEncoder(),
                                                  ['SexPclassAge'])]))])

In [37]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold()

In [40]:
cv.n_splits

5

In [41]:
import joblib

In [15]:
import json, yaml

with open("config.json", "w") as f:
    json.dump(config, fp=f)
# print(yaml.safe_dump(config))

In [17]:
with open("config.json", "r") as f:
    configload = json.load(f)

configload

{'experiment_name': '000_example',
 'data_prep': {'type': 'DynamicDataPrepPipeline',
  'params': {'ordinal_columns': ['Sex'],
   'numeric_columns': ['Age', 'Pclass', 'Fare'],
   'extract_title': True,
   'extract_fam': True,
   'extract_deck': True,
   'extract_sexpclassage': True}},
 'model': {'type': 'RandomForestClassifier',
  'params': {'max_depth': 20, 'random_state': 0}},
 'splitter': {'type': 'StratifiedKFold',
  'params': {'n_splits': 10, 'shuffle': True, 'random_state': 0}},
 'numeric_transformations': {'Fare': {'type': 'Pipeline',
   'steps': [{'name': 'bin',
     'transformer': {'type': 'KBinsDiscretizer',
      'params': {'n_bins': 4, 'encode': 'ordinal'}}},
    {'name': 'scale', 'transformer': {'type': 'MinMaxScaler'}}]},
  'Pclass': {'type': 'MinMaxScaler'}},
 'age_imputer_model': {'type': 'RandomForestRegressor',
  'params': {'max_depth': 20}}}

In [42]:
import joblib

pipeline = joblib.load(EXPERIMENTS_DIR / "001_tutorial_setup/ml_pipeline.joblib")

In [47]:
pipeline.named_steps["data_prep"].pipeline_

Pipeline(steps=[('extractor',
                 Pipeline(steps=[('fare_imputer', FareImputer()),
                                 ('fam', FamilySizeExtractor(extract=False)),
                                 ('title', TitleExtractor(extract=False)),
                                 ('deck', DeckExtractor(extract=False)),
                                 ('age_imputer', AgeImputer()),
                                 ('sexpclassage',
                                  SexPclassAgeExtractor(extract=False))])),
                ('col_tf',
                 ColumnTransformer(transformers=[('num_Parch', 'passthrough',
                                                  ['Parch']),
                                                 ('num_Pclass', 'passthrough',
                                                  ['Pclass']),
                                                 ('num_SibSp', 'passthrough',
                                                  ['SibSp']),
                                                 ('ord_Sex',
                                                  Pipeline(steps=[('encode',
                                                                   OrdinalEncoder()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['Sex'])]))])

In [55]:
import pandas as pd

grid_results = pd.read_csv(EXPERIMENTS_DIR / "002_hp_tune/grid_results.csv")
grid_results = pd.read_csv(EXPERIMENTS_DIR / "003_hp_tune_extractions/grid_results.csv")

In [57]:
pd.set_option("display.max_colwidth", 1000)
grid_results.sort_values(by="Test (Mean)", ascending=False)

,Run,Train (Mean),Train (Std),Test (Mean),Test (Std),Params
26,26,85.13,0.42,83.27,1.54,"{'data_prep__extract_deck': True, 'data_prep__extract_fam': True, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': True, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
30,30,84.26,0.36,83.16,1.49,"{'data_prep__extract_deck': True, 'data_prep__extract_fam': True, 'data_prep__extract_sexpclassage': True, 'data_prep__extract_title': True, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
10,10,84.99,0.44,83.05,1.62,"{'data_prep__extract_deck': False, 'data_prep__extract_fam': True, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': True, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
8,8,85.55,0.36,83.05,1.46,"{'data_prep__extract_deck': False, 'data_prep__extract_fam': True, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': False, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
14,14,84.37,0.34,82.82,1.48,"{'data_prep__extract_deck': False, 'data_prep__extract_fam': True, 'data_prep__extract_sexpclassage': True, 'data_prep__extract_title': True, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
2,2,86.08,0.21,82.60,1.46,"{'data_prep__extract_deck': False, 'data_prep__extract_fam': False, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': True, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
0,0,86.76,0.22,82.49,1.64,"{'data_prep__extract_deck': False, 'data_prep__extract_fam': False, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': False, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
16,16,87.71,0.31,82.49,1.36,"{'data_prep__extract_deck': True, 'data_prep__extract_fam': False, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': False, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
12,12,84.82,0.32,82.38,1.50,"{'data_prep__extract_deck': False, 'data_prep__extract_fam': True, 'data_prep__extract_sexpclassage': True, 'data_prep__extract_title': False, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
18,18,86.64,0.29,82.26,1.65,"{'data_prep__extract_deck': True, 'data_prep__extract_fam': False, 'data_prep__extract_sexpclassage': False, 'data_prep__extract_title': True, 'data_prep__numeric_columns': ['Age', 'Pclass', 'Fare'], 'data_prep__ordinal_columns': ['Sex'], 'model__max_depth': 5}"
